# Preprocess ERA 5 data seasonal climatology
Creates seasonal climatology for ERA 5 data from the preproccessed monthly average files

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Set in paths

In [2]:
# data in, preprocessed monthly means
base = '/work/Marion.Alberty/data/ERA5_pp/'
# u-winds
u10_dir = base + 'u_10m/'
# v-winds
v10_dir = base + 'v_10m/'
# sea level pressure
slp_dir = base + 'P_mean_sea_level/'

## Set year range for climatologies

In [3]:
ystart = '1979'
yend = '2014'

## Climatology algorithm

In [4]:
def preproc_ERA5_climatology(in_dir,i_year,n_year):
    # import pkgs
    from glob import glob
    import xarray as xr
    from dask.diagnostics import ProgressBar
    import numpy as np
    
    # Parse in_dir to get varaible
    var_dir = in_dir.split('/')[-2]
    
    # climatologies out
    out_dir = '/work/Marion.Alberty/data/ERA5_pp/'
    
    # Get list of ERA5 files
    flist = glob(f'{in_dir}*.nc')
    
    # Read in data and calculate the monthly averages
    ds = xr.open_mfdataset(flist,use_cftime=True,engine='netcdf4'
                          ).chunk({'time':24}
                                 ).sel(time=slice(i_year,n_year)
                                      ).groupby('time.month').mean().chunk({'month':12})
    
    # Add attributes
    _ = ds.attrs['description'] = "ERA5 seasonal climatology"
    _ = ds.attrs['climatology_period'] = i_year + " to " + n_year
    
    # Write climatology to netcdf
    with ProgressBar():
        ds.to_netcdf(out_dir + 'ERA5_climatology_' + var_dir + '_' + i_year + '_' + n_year + '.nc')

## Set up DASK cluster
In dask labextension, create a +NEW cluster. Click the < > widget, scale it to 16 workers and let's go!

## Process and write variables

In [5]:
# u-winds
preproc_ERA5_climatology(u10_dir,ystart,yend)

[########################################] | 100% Completed |  4.4s


In [6]:
# v-winds
preproc_ERA5_climatology(v10_dir,ystart,yend)

[########################################] | 100% Completed |  5.3s


In [7]:
# sea level pressure
preproc_ERA5_climatology(slp_dir,ystart,yend)

[########################################] | 100% Completed |  5.1s
